In [8]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [9]:
DATA_DIR = '/mnt/chicm/data/open-images/relation'

In [10]:
def get_top_classes(start_index=0, end_index=57):
    df = pd.read_csv(os.path.join(DATA_DIR, 'top_classes.csv'))
    #print(df.shape)
    c = df['class'].values[start_index:end_index]
    #print(df.head())
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi


In [11]:
classes, _ = get_top_classes()
classes[:5]

array(['/m/04yx4', '/m/03bt1vf', '/m/0k4j', '/m/05r655', '/m/01mzpv'],
      dtype=object)

In [12]:
classes_57 = set(classes)

In [13]:
def get_det(pred_str):
    dets = []
    det = []
    for i, e in enumerate(pred_str.split(' ')):
        if i % 6 == 0:
            det = []
        det.append(e)
        if (i+1) % 6 == 0:
            if det[0] in classes_57: #and float(det[1]) > 0.1:
                dets.append(det)
            
    return dets

In [14]:
!ls

all_models_team_submission_v1_large_20190813.csv
all_models_team_submission_v1_large_20190813.csv.zip
catboost_info
cat_feature_index.c1a4dabd-715a703f-75517141-d81266d1.tmp
cat_feature_index.e36fdba6-6cd399f6-229ea45f-ea07b906.tmp
detect-57-ensemble.ipynb
eda.ipynb
ensemble_detect_57_0808_1.csv
ensemble_detect_57_0808_1_top50.csv
ensemble_detect_57_0808_2_with_weights.csv
ensemble_detect_57_0819_1.csv
ensemble_detect_57_0819_2.csv
ensemble_detect_57_0820_1_top100.csv
ensemble_is_42_0808_1_3_models.csv
filter_detect_submission.ipynb
Is-classifier-6-eda.ipynb
Is-detect-42-eda.ipynb
lb18901
lb19252
lb26760
lb28188
merge_0808_1.csv.zip
merge_0808_2.csv.zip
merge_0820_1.csv.zip
merge_0821_1.csv.zip
merge_df.ipynb
sub_detect_57
sub_is_42
submit-detect-57.ipynb
submit-is-42-ensemble.ipynb
submit-is-42.ipynb
submit-is-6-catboost.ipynb
submit-is-6.ipynb
subs
sub_sort_200_0808_1_fast_ensemble_detect.csv
sub_sort_200_0808_2_fast_ensemble_detect_weights.csv
sub_sort_200_0819_2_fast_ensemble.csv
s

In [15]:
df_det = pd.read_csv('all_models_team_submission_v1_large_20190813.csv')
df_det.head()

,ImageId,PredictionString
0,00000b4dcff7f799,/m/04yx4 0.528 0.860 0.305 0.956 0.811 /m/04yx...
1,00001a21632de752,/m/04yx4 0.512 0.007 0.491 0.188 0.935 /m/04yx...
2,0000d67245642c5f,/m/0d4v4 0.112 0.516 0.214 0.544 0.259 /m/0d4v...
3,0001244aa8ed3099,/m/04yx4 0.004 0.898 0.295 0.999 0.535 /m/04yx...
4,000172d1dd1adce0,/m/04yx4 0.005 0.638 0.021 0.687 0.188 /m/0d4v...


In [16]:
df_det['dets'] = df_det.PredictionString.map(lambda x: get_det(str(x)))
df_det.head()

,ImageId,PredictionString,dets
0,00000b4dcff7f799,/m/04yx4 0.528 0.860 0.305 0.956 0.811 /m/04yx...,"[[/m/04yx4, 0.528, 0.860, 0.305, 0.956, 0.811]..."
1,00001a21632de752,/m/04yx4 0.512 0.007 0.491 0.188 0.935 /m/04yx...,"[[/m/04yx4, 0.512, 0.007, 0.491, 0.188, 0.935]..."
2,0000d67245642c5f,/m/0d4v4 0.112 0.516 0.214 0.544 0.259 /m/0d4v...,"[[/m/05r5c, 0.026, 0.147, 0.011, 1.000, 1.000]]"
3,0001244aa8ed3099,/m/04yx4 0.004 0.898 0.295 0.999 0.535 /m/04yx...,"[[/m/04yx4, 0.004, 0.898, 0.295, 0.999, 0.535]..."
4,000172d1dd1adce0,/m/04yx4 0.005 0.638 0.021 0.687 0.188 /m/0d4v...,"[[/m/04yx4, 0.005, 0.638, 0.021, 0.687, 0.188]..."


In [30]:
df_det.dets.map(lambda x: len(x)).values.sum()

11084973

In [31]:
df_det.dets.map(lambda x: len(x)).values.min()

0

In [32]:
def get_pred_str(dets):
    if len(dets) < 1:
        return ''
    res = []
    dets = sorted(dets, key=lambda x: x[1], reverse=True)[:150]
    for det in dets:
        res.extend(det)
        
    return ' '.join(res)

In [33]:
df_det['PredictionString'] = df_det.dets.map(lambda x: get_pred_str(x))
df_det.head()

,ImageId,PredictionString,dets
0,00000b4dcff7f799,/m/04yx4 0.528 0.860 0.305 0.956 0.811 /m/04yx...,"[[/m/04yx4, 0.528, 0.860, 0.305, 0.956, 0.811]..."
1,00001a21632de752,/m/04yx4 0.679 0.613 0.494 0.760 0.731 /m/03bt...,"[[/m/04yx4, 0.512, 0.007, 0.491, 0.188, 0.935]..."
2,0000d67245642c5f,/m/05r5c 0.026 0.147 0.011 1.000 1.000,"[[/m/05r5c, 0.026, 0.147, 0.011, 1.000, 1.000]]"
3,0001244aa8ed3099,/m/05ctyq 0.035 0.556 0.369 0.614 0.457 /m/0k4...,"[[/m/04yx4, 0.004, 0.898, 0.295, 0.999, 0.535]..."
4,000172d1dd1adce0,/m/0bt9lr 0.021 0.253 0.061 0.785 0.813 /m/04y...,"[[/m/04yx4, 0.005, 0.638, 0.021, 0.687, 0.188]..."


In [34]:
df_det.to_csv('sub_filter_57_top150.csv', columns=['ImageId', 'PredictionString'], index=False)

In [35]:
!kaggle competitions submit -c open-images-2019-object-detection -f sub_filter_57_top150.csv -m "submit"

100%|████████████████████████████████████████| 297M/297M [00:18<00:00, 16.5MB/s]
Successfully submitted to Open Images 2019 - Object Detection